In [2]:
"""
Knowledge Graph Builder for Energy District Analysis - PRE-GNN VERSION
Updated to use actual PostgreSQL database schema with rich grid infrastructure
Creates the foundational KG with raw data, infrastructure, and potential
Complementarity and clustering will be added AFTER GNN analysis
"""

import pandas as pd
import numpy as np
from neo4j import GraphDatabase
import psycopg2
from psycopg2.extras import RealDictCursor
from datetime import datetime, timedelta
import json
import logging
from pathlib import Path
from typing import Dict, List, Tuple, Any
import warnings
warnings.filterwarnings('ignore')
from decimal import Decimal

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class EnergyKnowledgeGraphBuilder:
    """Build Knowledge Graph from energy district data - Pre-GNN version"""
    
    def __init__(self, neo4j_uri: str, neo4j_user: str, neo4j_password: str,
                 pg_host: str, pg_database: str, pg_user: str, pg_password: str):
        """Initialize Neo4j and PostgreSQL connections"""
        # Neo4j connection
        self.driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))
        
        # PostgreSQL connection
        self.pg_conn = psycopg2.connect(
            host=pg_host,
            database=pg_database,
            user=pg_user,
            password=pg_password,
            port=5433
        )
        
        self.stats = {
            'nodes_created': {},
            'relationships_created': {},
            'processing_time': {}
        }
        logger.info(f"Connected to Neo4j at {neo4j_uri}")
        logger.info(f"Connected to PostgreSQL database {pg_database}")
    
    def close(self):
        """Close database connections"""
        self.driver.close()
        self.pg_conn.close()
        logger.info("Database connections closed")
    
    def clear_database(self):
        """Clear all nodes and relationships"""
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")
            logger.info("Neo4j database cleared")
    
    def convert_decimals(self, record):
        """Convert Decimal types to float for Neo4j compatibility"""
        if isinstance(record, dict):
            return {k: float(v) if isinstance(v, Decimal) else v for k, v in record.items()}
        elif isinstance(record, list):
            return [self.convert_decimals(item) for item in record]
        else:
            return float(record) if isinstance(record, Decimal) else record

    # ============================================
    # STEP 1: SCHEMA SETUP
    # ============================================
    
    def create_schema(self):
        """Create constraints and indexes for pre-GNN KG"""
        logger.info("Creating schema constraints and indexes...")
        
        constraints = [
            # Infrastructure
            "CREATE CONSTRAINT IF NOT EXISTS FOR (s:Substation) REQUIRE s.station_id IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (t:Transformer) REQUIRE t.transformer_id IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (c:LVCabinet) REQUIRE c.cabinet_id IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (g:CableGroup) REQUIRE g.group_id IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (seg:CableSegment) REQUIRE seg.segment_id IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (b:Building) REQUIRE b.ogc_fid IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (cp:ConnectionPoint) REQUIRE cp.point_id IS UNIQUE",
            
            # Temporal
            "CREATE CONSTRAINT IF NOT EXISTS FOR (t:TimeSlot) REQUIRE t.slot_id IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (e:EnergyState) REQUIRE e.state_id IS UNIQUE",
            
            # Assets
            "CREATE CONSTRAINT IF NOT EXISTS FOR (sol:SolarSystem) REQUIRE sol.system_id IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (bat:BatterySystem) REQUIRE bat.system_id IS UNIQUE",
            "CREATE CONSTRAINT IF NOT EXISTS FOR (hp:HeatPumpSystem) REQUIRE hp.system_id IS UNIQUE",
        ]
        
        indexes = [
            # Performance indexes
            "CREATE INDEX IF NOT EXISTS FOR (g:CableGroup) ON (g.voltage_level)",
            "CREATE INDEX IF NOT EXISTS FOR (b:Building) ON (b.lv_group_id)",
            "CREATE INDEX IF NOT EXISTS FOR (b:Building) ON (b.building_function)",
            "CREATE INDEX IF NOT EXISTS FOR (b:Building) ON (b.district_name)",
            "CREATE INDEX IF NOT EXISTS FOR (b:Building) ON (b.has_solar)",
            "CREATE INDEX IF NOT EXISTS FOR (seg:CableSegment) ON (seg.group_id)",
            "CREATE INDEX IF NOT EXISTS FOR (cp:ConnectionPoint) ON (cp.building_id)",
            "CREATE INDEX IF NOT EXISTS FOR (t:TimeSlot) ON (t.timestamp)",
            "CREATE INDEX IF NOT EXISTS FOR (t:TimeSlot) ON (t.hour_of_day)",
            "CREATE INDEX IF NOT EXISTS FOR (e:EnergyState) ON (e.building_id)",
        ]
        
        with self.driver.session() as session:
            for constraint in constraints:
                session.run(constraint)
            for index in indexes:
                session.run(index)
        
        logger.info("Schema created successfully")
    
    # ============================================
    # STEP 2: LOAD GRID INFRASTRUCTURE
    # ============================================
    
    def load_grid_infrastructure(self):
        """Load complete grid topology from PostgreSQL"""
        logger.info("Loading grid infrastructure from PostgreSQL...")
        
        with self.pg_conn.cursor(cursor_factory=RealDictCursor) as cursor:
            
            # Load Substations
            cursor.execute("""
                SELECT 
                    fid as station_id,
                    ST_X(clipped_geom) as x,
                    ST_Y(clipped_geom) as y,
                    ST_AsText(clipped_geom) as geom_wkt
                FROM amin_grid.tlip_onderstations
                WHERE clipped_geom IS NOT NULL
            """)
            substations = [self.convert_decimals(row) for row in cursor.fetchall()]
            
            # Load Transformers (MV installations)
            cursor.execute("""
                SELECT 
                    fid as transformer_id,
                    ST_X(clipped_geom) as x,
                    ST_Y(clipped_geom) as y,
                    ST_AsText(clipped_geom) as geom_wkt
                FROM amin_grid.tlip_middenspanningsinstallaties
                WHERE clipped_geom IS NOT NULL
            """)
            transformers = [self.convert_decimals(row) for row in cursor.fetchall()]
            
            # Load LV Cabinets
            cursor.execute("""
                SELECT 
                    fid as cabinet_id,
                    ST_X(clipped_geom) as x,
                    ST_Y(clipped_geom) as y,
                    ST_AsText(clipped_geom) as geom_wkt
                FROM amin_grid.tlip_laagspanningsverdeelkasten
                WHERE clipped_geom IS NOT NULL
            """)
            lv_cabinets = [self.convert_decimals(row) for row in cursor.fetchall()]
            
            # Load Cable Groups (connected components)
            cursor.execute("""
                SELECT 
                    group_id,
                    voltage_level,
                    segment_count,
                    total_length_m,
                    ST_X(centroid) as x,
                    ST_Y(centroid) as y,
                    ST_AsText(bbox) as bbox_wkt
                FROM amin_grid.tlip_connected_groups
            """)
            cable_groups = [self.convert_decimals(row) for row in cursor.fetchall()]
            
            # Load Cable Segments
            cursor.execute("""
                SELECT 
                    segment_id,
                    original_fid,
                    voltage_level,
                    group_id,
                    length_m,
                    ST_X(start_point) as start_x,
                    ST_Y(start_point) as start_y,
                    ST_X(end_point) as end_x,
                    ST_Y(end_point) as end_y
                FROM amin_grid.tlip_cable_segments
                WHERE group_id IS NOT NULL
            """)
            cable_segments = [self.convert_decimals(row) for row in cursor.fetchall()]
            
            # Load hierarchy relationships
            cursor.execute("""
                SELECT 
                    child_group_id,
                    child_voltage,
                    parent_group_id,
                    parent_voltage,
                    connection_via,
                    via_station_fid,
                    confidence_score
                FROM amin_grid.tlip_group_hierarchy
            """)
            hierarchy = [self.convert_decimals(row) for row in cursor.fetchall()]
            
            # Load group-station connections
            cursor.execute("""
                SELECT 
                    group_id,
                    voltage_level,
                    station_type,
                    station_fid,
                    connection_type,
                    distance_m,
                    confidence_score
                FROM amin_grid.tlip_group_stations
            """)
            group_stations = [self.convert_decimals(row) for row in cursor.fetchall()]
        
        # Create nodes in Neo4j
        with self.driver.session() as session:
            # Create Substations
            for substation in substations:
                session.run("""
                    CREATE (s:GridComponent:Substation {
                        station_id: $station_id,
                        x: $x,
                        y: $y,
                        voltage_level: 'HV',
                        component_type: 'substation',
                        geom_wkt: $geom_wkt
                    })
                """, **substation)
            self.stats['nodes_created']['Substation'] = len(substations)
            
            # Create Transformers
            for transformer in transformers:
                session.run("""
                    CREATE (t:GridComponent:Transformer {
                        transformer_id: $transformer_id,
                        x: $x,
                        y: $y,
                        voltage_level: 'MV',
                        component_type: 'transformer',
                        geom_wkt: $geom_wkt
                    })
                """, **transformer)
            self.stats['nodes_created']['Transformer'] = len(transformers)
            
            # Create LV Cabinets
            for cabinet in lv_cabinets:
                session.run("""
                    CREATE (c:GridComponent:LVCabinet {
                        cabinet_id: $cabinet_id,
                        x: $x,
                        y: $y,
                        voltage_level: 'LV',
                        component_type: 'lv_cabinet',
                        geom_wkt: $geom_wkt
                    })
                """, **cabinet)
            self.stats['nodes_created']['LVCabinet'] = len(lv_cabinets)
            
            # Create Cable Groups
            for group in cable_groups:
                session.run("""
                    CREATE (g:GridComponent:CableGroup {
                        group_id: $group_id,
                        voltage_level: $voltage_level,
                        segment_count: $segment_count,
                        total_length_m: $total_length_m,
                        x: $x,
                        y: $y,
                        bbox_wkt: $bbox_wkt,
                        component_type: 'cable_group'
                    })
                """, **group)
            self.stats['nodes_created']['CableGroup'] = len(cable_groups)
            
            # Create Cable Segments
            segment_batch = []
            for segment in cable_segments:
                segment_batch.append({
                    'segment_id': segment['segment_id'],
                    'original_fid': segment['original_fid'],
                    'voltage_level': segment['voltage_level'],
                    'group_id': segment['group_id'],
                    'length_m': float(segment['length_m']),
                    'start_x': float(segment['start_x']),
                    'start_y': float(segment['start_y']),
                    'end_x': float(segment['end_x']),
                    'end_y': float(segment['end_y'])
                })
                
                if len(segment_batch) >= 1000:
                    session.run("""
                        UNWIND $segments as seg
                        CREATE (s:CableSegment {
                            segment_id: seg.segment_id,
                            original_fid: seg.original_fid,
                            voltage_level: seg.voltage_level,
                            group_id: seg.group_id,
                            length_m: seg.length_m,
                            start_x: seg.start_x,
                            start_y: seg.start_y,
                            end_x: seg.end_x,
                            end_y: seg.end_y
                        })
                    """, segments=segment_batch)
                    segment_batch = []
            
            if segment_batch:
                session.run("""
                    UNWIND $segments as seg
                    CREATE (s:CableSegment {
                        segment_id: seg.segment_id,
                        original_fid: seg.original_fid,
                        voltage_level: seg.voltage_level,
                        group_id: seg.group_id,
                        length_m: seg.length_m,
                        start_x: seg.start_x,
                        start_y: seg.start_y,
                        end_x: seg.end_x,
                        end_y: seg.end_y
                    })
                """, segments=segment_batch)
            
            self.stats['nodes_created']['CableSegment'] = len(cable_segments)
            
            # Create hierarchy relationships
            for rel in hierarchy:
                session.run("""
                    MATCH (child:CableGroup {group_id: $child_group_id})
                    MATCH (parent:CableGroup {group_id: $parent_group_id})
                    CREATE (child)-[:FEEDS_FROM {
                        connection_via: $connection_via,
                        via_station_fid: $via_station_fid,
                        confidence_score: $confidence_score
                    }]->(parent)
                """, **rel)
            self.stats['relationships_created']['FEEDS_FROM'] = len(hierarchy)
            
            # Create group-station relationships
            for conn in group_stations:
                if conn['station_type'] == 'TRANSFORMER':
                    session.run("""
                        MATCH (g:CableGroup {group_id: $group_id})
                        MATCH (t:Transformer {transformer_id: $station_fid})
                        CREATE (g)-[:CONNECTS_TO {
                            connection_type: $connection_type,
                            distance_m: $distance_m,
                            confidence_score: $confidence_score
                        }]->(t)
                    """, **conn)
                elif conn['station_type'] == 'SUBSTATION':
                    session.run("""
                        MATCH (g:CableGroup {group_id: $group_id})
                        MATCH (s:Substation {station_id: $station_fid})
                        CREATE (g)-[:CONNECTS_TO {
                            connection_type: $connection_type,
                            distance_m: $distance_m,
                            confidence_score: $confidence_score
                        }]->(s)
                    """, **conn)
                elif conn['station_type'] == 'LV_CABINET':
                    session.run("""
                        MATCH (g:CableGroup {group_id: $group_id})
                        MATCH (c:LVCabinet {cabinet_id: $station_fid})
                        CREATE (g)-[:CONNECTS_TO {
                            connection_type: $connection_type,
                            distance_m: $distance_m,
                            confidence_score: $confidence_score
                        }]->(c)
                    """, **conn)
            
            # Create segment to group relationships
            session.run("""
                MATCH (s:CableSegment)
                MATCH (g:CableGroup {group_id: s.group_id})
                CREATE (s)-[:PART_OF]->(g)
            """)
            
        logger.info(f"Created {sum(self.stats['nodes_created'].values())} infrastructure nodes")
    
    # ============================================
    # STEP 3: LOAD AND ENHANCE BUILDINGS
    # ============================================
    
    def load_buildings(self):
        """Load buildings with all connection metadata"""
        logger.info("Loading building data from PostgreSQL...")
        
        with self.pg_conn.cursor(cursor_factory=RealDictCursor) as cursor:
            # Load buildings with connection data
            cursor.execute("""
                SELECT 
                    b.ogc_fid,
                    b.x,
                    b.y,
                    b.building_function,
                    b.residential_type,
                    b.non_residential_type,
                    b.area,
                    b.height,
                    b.age_range,
                    b.building_orientation_cardinal,
                    b.meestvoorkomendelabel as energy_label,
                    b.woningtype as housing_type,
                    b.wijknaam as district_name,
                    b.buurtnaam as neighborhood_name,
                    b.bouwjaar as building_year,
                    b.b3_opp_dak_plat as flat_roof_area,
                    b.b3_opp_dak_schuin as sloped_roof_area,
                    -- Connection data
                    bc.connected_group_id as lv_group_id,
                    bc.connected_segment_id,
                    bc.connection_distance_m,
                    bc.connection_type,
                    bc.is_mv_capable,
                    bc.has_mv_nearby,
                    bc.nearest_mv_distance_m,
                    bc.is_problematic,
                    bc.connection_reason,
                    -- Energy indicators
                    b.ndvi_mean_100m,
                    b.ntl_mean_500m,
                    b.ndwi_mean_250m
                FROM amin.buildings_1_deducted b
                LEFT JOIN amin_grid.tlip_building_connections bc
                    ON b.ogc_fid = bc.building_id
                WHERE b.area > 10 
                    AND b.pand_geom IS NOT NULL
                    AND bc.connected_group_id IS NOT NULL
            """)
            buildings = [self.convert_decimals(row) for row in cursor.fetchall()]
            
            # Load connection points
            cursor.execute("""
                SELECT 
                    connection_point_id as point_id,
                    building_id,
                    segment_id,
                    group_id,
                    connection_type,
                    ST_X(point_on_line) as point_x,
                    ST_Y(point_on_line) as point_y,
                    distance_along_segment,
                    segment_fraction,
                    connection_distance_m,
                    is_direct_connection
                FROM amin_grid.tlip_building_connection_points
            """)
            connection_points = [self.convert_decimals(row) for row in cursor.fetchall()]
        
        # Process buildings and add derived features
        buildings_df = pd.DataFrame(buildings)
        buildings_df = self._calculate_building_features(buildings_df)
        
        # Create nodes in Neo4j
        with self.driver.session() as session:
            # Create building nodes with all features
            for _, building in buildings_df.iterrows():
                session.run("""
                    CREATE (b:Building {
                        ogc_fid: $ogc_fid,
                        x: $x,
                        y: $y,
                        building_function: $function,
                        residential_type: $res_type,
                        non_residential_type: $non_res_type,
                        area: $area,
                        height: $height,
                        age_range: $age,
                        building_year: $year,
                        building_orientation_cardinal: $orientation,
                        district_name: $district,
                        neighborhood_name: $neighborhood,
                        housing_type: $housing_type,
                        
                        // Roof data
                        flat_roof_area: $flat_roof,
                        sloped_roof_area: $sloped_roof,
                        suitable_roof_area: $suitable_roof,
                        
                        // Connection data
                        lv_group_id: $lv_group,
                        connection_segment_id: $segment_id,
                        connection_type: $conn_type,
                        connection_distance_m: $conn_distance,
                        is_mv_capable: $mv_capable,
                        has_mv_nearby: $mv_nearby,
                        nearest_mv_distance_m: $mv_distance,
                        is_problematic: $problematic,
                        connection_reason: $conn_reason,
                        
                        // Energy features
                        energy_label: $energy_label,
                        energy_label_simple: $energy_label_simple,
                        insulation_quality: $insulation,
                        solar_potential: $solar_pot,
                        solar_capacity_kwp: $solar_kwp,
                        battery_readiness: $battery_ready,
                        electrification_feasibility: $elec_feasible,
                        expected_cop: $cop,
                        
                        // Environmental indicators
                        vegetation_index: $ndvi,
                        nighttime_lights: $ntl,
                        water_index: $ndwi,
                        
                        // Current assets (initially false/none)
                        has_solar: $has_solar,
                        has_battery: $has_battery,
                        has_heat_pump: $has_hp,
                        heating_system: $heating
                    })
                """,
                ogc_fid=int(building['ogc_fid']),
                x=float(building['x']),
                y=float(building['y']),
                function=building['building_function'],
                res_type=building['residential_type'] if pd.notna(building['residential_type']) else 'None',
                non_res_type=building['non_residential_type'] if pd.notna(building['non_residential_type']) else 'None',
                area=float(building['area']),
                height=float(building['height']) if pd.notna(building['height']) else 3.0,
                age=building['age_range'] if pd.notna(building['age_range']) else 'Unknown',
                year=int(building['building_year']) if pd.notna(building['building_year']) else 0,
                orientation=building['building_orientation_cardinal'] if pd.notna(building['building_orientation_cardinal']) else 'Unknown',
                district=building['district_name'] if pd.notna(building['district_name']) else 'Unknown',
                neighborhood=building['neighborhood_name'] if pd.notna(building['neighborhood_name']) else 'Unknown',
                housing_type=building['housing_type'] if pd.notna(building['housing_type']) else 'Unknown',
                flat_roof=float(building['flat_roof_area']) if pd.notna(building['flat_roof_area']) else 0.0,
                sloped_roof=float(building['sloped_roof_area']) if pd.notna(building['sloped_roof_area']) else 0.0,
                suitable_roof=float(building['suitable_roof_area']),
                lv_group=building['lv_group_id'],
                segment_id=int(building['connected_segment_id']) if pd.notna(building['connected_segment_id']) else 0,
                conn_type=building['connection_type'],
                conn_distance=float(building['connection_distance_m']),
                mv_capable=bool(building['is_mv_capable']),
                mv_nearby=bool(building['has_mv_nearby']),
                mv_distance=float(building['nearest_mv_distance_m']) if pd.notna(building['nearest_mv_distance_m']) else 999.0,
                problematic=bool(building['is_problematic']),
                conn_reason=building['connection_reason'] if pd.notna(building['connection_reason']) else '',
                energy_label=building['energy_label'] if pd.notna(building['energy_label']) else 'Unknown',
                energy_label_simple=building['energy_label_simple'],
                insulation=building['insulation_quality'],
                solar_pot=building['solar_potential'],
                solar_kwp=float(building['solar_capacity_kwp']),
                battery_ready=building['battery_readiness'],
                elec_feasible=building['electrification_feasibility'],
                cop=float(building['expected_cop']),
                ndvi=float(building['ndvi_mean_100m']) if pd.notna(building['ndvi_mean_100m']) else 0.0,
                ntl=float(building['ntl_mean_500m']) if pd.notna(building['ntl_mean_500m']) else 0.0,
                ndwi=float(building['ndwi_mean_250m']) if pd.notna(building['ndwi_mean_250m']) else 0.0,
                has_solar=bool(building['has_solar']),
                has_battery=bool(building['has_battery']),
                has_hp=bool(building['has_heat_pump']),
                heating=building['heating_system']
                )
            
            self.stats['nodes_created']['Building'] = len(buildings_df)
            
            # Create Connection Point nodes
            for point in connection_points:
                session.run("""
                    CREATE (cp:ConnectionPoint {
                        point_id: $point_id,
                        building_id: $building_id,
                        segment_id: $segment_id,
                        group_id: $group_id,
                        connection_type: $conn_type,
                        x: $x,
                        y: $y,
                        distance_along_segment: $dist_along,
                        segment_fraction: $fraction,
                        connection_distance_m: $conn_dist,
                        is_direct: $is_direct
                    })
                """,
                point_id=int(point['point_id']),
                building_id=int(point['building_id']),
                segment_id=int(point['segment_id']),
                group_id=point['group_id'],
                conn_type=point['connection_type'],
                x=float(point['point_x']),
                y=float(point['point_y']),
                dist_along=float(point['distance_along_segment']),
                fraction=float(point['segment_fraction']),
                conn_dist=float(point['connection_distance_m']),
                is_direct=bool(point['is_direct_connection'])
                )
            
            self.stats['nodes_created']['ConnectionPoint'] = len(connection_points)
            
            # Create relationships
            # Building -> LV Cable Group
            result = session.run("""
                MATCH (b:Building)
                MATCH (g:CableGroup {group_id: b.lv_group_id})
                CREATE (b)-[:CONNECTED_TO {
                    connection_type: b.connection_type,
                    distance_m: b.connection_distance_m,
                    is_problematic: b.is_problematic
                }]->(g)
                RETURN count(*) as count
            """)
            self.stats['relationships_created']['CONNECTED_TO'] = result.single()['count']
            
            # Building -> Connection Point
            result = session.run("""
                MATCH (b:Building)
                MATCH (cp:ConnectionPoint {building_id: b.ogc_fid})
                CREATE (b)-[:HAS_CONNECTION_POINT]->(cp)
                RETURN count(*) as count
            """)
            self.stats['relationships_created']['HAS_CONNECTION_POINT'] = result.single()['count']
            
            # Connection Point -> Cable Segment
            result = session.run("""
                MATCH (cp:ConnectionPoint)
                MATCH (s:CableSegment {segment_id: cp.segment_id})
                CREATE (cp)-[:ON_SEGMENT {
                    fraction: cp.segment_fraction,
                    distance_along: cp.distance_along_segment
                }]->(s)
                RETURN count(*) as count
            """)
            self.stats['relationships_created']['ON_SEGMENT'] = result.single()['count']
            
            # Building NEAR_MV for MV-capable buildings
            # Building NEAR_MV for MV-capable buildings
            # Building NEAR_MV for MV-capable buildings
            result = session.run("""
                MATCH (b:Building {is_mv_capable: true, has_mv_nearby: true})
                MATCH (g:CableGroup {voltage_level: 'MV'})
                WHERE point.distance(point({x: b.x, y: b.y}), point({x: g.x, y: g.y})) < 200
                WITH b, g, point.distance(point({x: b.x, y: b.y}), point({x: g.x, y: g.y})) as dist
                ORDER BY dist
                WITH b, head(collect(g)) as nearest_mv, min(dist) as min_dist
                CREATE (b)-[:NEAR_MV {distance_m: min_dist}]->(nearest_mv)
                RETURN count(*) as count
            """)
            self.stats['relationships_created']['NEAR_MV'] = result.single()['count']
            
        logger.info(f"Created {len(buildings_df)} building nodes with {len(connection_points)} connection points")
    
    def _calculate_building_features(self, buildings: pd.DataFrame) -> pd.DataFrame:
        """Calculate derived building features"""
        
        # Simplify energy label
        def simplify_label(label):
            if pd.isna(label):
                return 'Unknown'
            if label in ['A', 'A+', 'A++', 'A+++', 'A++++']:
                return 'A'
            elif label in ['B', 'C', 'D', 'E', 'F', 'G']:
                return label
            else:
                return 'Unknown'
        
        buildings['energy_label_simple'] = buildings['energy_label'].apply(simplify_label)
        
        # If no energy label, estimate from age
        def estimate_label(row):
            if row['energy_label_simple'] != 'Unknown':
                return row['energy_label_simple']
            
            age_labels = {
                '< 1945': 'F',
                '1945-1975': 'E', 
                '1975-1990': 'D',
                '1990-2005': 'C',
                '2005-2015': 'B',
                '> 2015': 'A'
            }
            return age_labels.get(row['age_range'], 'D')
        
        buildings['energy_label_simple'] = buildings.apply(estimate_label, axis=1)
        
        # Insulation quality
        label_insulation = {
            'A': 'excellent', 'B': 'good', 'C': 'fair',
            'D': 'fair', 'E': 'poor', 'F': 'poor', 
            'G': 'very_poor', 'Unknown': 'unknown'
        }
        buildings['insulation_quality'] = buildings['energy_label_simple'].map(label_insulation)
        
        # Calculate suitable roof area
        buildings['suitable_roof_area'] = buildings['flat_roof_area'].fillna(0) + \
            buildings.apply(lambda x: x['sloped_roof_area'] * 0.6 
                          if x['building_orientation_cardinal'] in ['S', 'SE', 'SW'] 
                          else x['sloped_roof_area'] * 0.3 if pd.notna(x['sloped_roof_area']) else 0, axis=1)
        
        # Solar potential
        def get_solar_potential(row):
            if row['suitable_roof_area'] > 100:
                return 'high'
            elif row['suitable_roof_area'] > 50:
                return 'medium'
            elif row['suitable_roof_area'] > 25:
                return 'low'
            else:
                return 'none'
        
        buildings['solar_potential'] = buildings.apply(get_solar_potential, axis=1)
        
        # Solar capacity potential
        buildings['solar_capacity_kwp'] = buildings['suitable_roof_area'] * 0.15 * 0.7
        
        # Battery readiness (based on solar potential and building type)
        def get_battery_readiness(row):
            if row['solar_potential'] in ['high', 'medium']:
                return 'ready'
            elif row['is_mv_capable'] or row['building_function'] == 'non_residential':
                return 'conditional'
            else:
                return 'not_ready'
        
        buildings['battery_readiness'] = buildings.apply(get_battery_readiness, axis=1)
        
        # Electrification feasibility
        def get_electrification_feasibility(row):
            if row['energy_label_simple'] in ['F', 'G']:
                return 'upgrade_needed'
            elif row['energy_label_simple'] in ['D', 'E']:
                return 'conditional'
            else:
                return 'immediate'
        
        buildings['electrification_feasibility'] = buildings.apply(get_electrification_feasibility, axis=1)
        
        # Expected COP for heat pumps
        cop_by_label = {
            'A': 4.0, 'B': 3.5, 'C': 3.0,
            'D': 2.5, 'E': 2.0, 'F': 1.8, 
            'G': 1.5, 'Unknown': 2.0
        }
        buildings['expected_cop'] = buildings['energy_label_simple'].map(cop_by_label)
        
        # Initial asset assignment (most buildings don't have assets yet)
        buildings['has_solar'] = False
        buildings['has_battery'] = False
        buildings['has_heat_pump'] = (buildings['energy_label_simple'].isin(['A', 'B'])) & \
                                     (np.random.random(len(buildings)) > 0.8)
        buildings['heating_system'] = buildings.apply(
            lambda x: 'heat_pump' if x['has_heat_pump'] else 'gas', axis=1
        )
        
        # Some buildings might already have solar (simulate 5% adoption)
        high_potential_mask = buildings['solar_potential'].isin(['high', 'medium'])
        buildings.loc[high_potential_mask, 'has_solar'] = np.random.random(high_potential_mask.sum()) > 0.95
        
        # Batteries only for buildings with solar
        buildings.loc[buildings['has_solar'], 'has_battery'] = np.random.random(buildings['has_solar'].sum()) > 0.7
        
        logger.info(f"Building features calculated: {len(buildings)} buildings processed")
        logger.info(f"  Solar potential - High: {(buildings['solar_potential']=='high').sum()}, "
                   f"Medium: {(buildings['solar_potential']=='medium').sum()}")
        logger.info(f"  Existing installations - Solar: {buildings['has_solar'].sum()}, "
                   f"Battery: {buildings['has_battery'].sum()}, "
                   f"Heat Pump: {buildings['has_heat_pump'].sum()}")
        
        return buildings
    
    # ============================================
    # STEP 4: CREATE EXISTING ASSET NODES
    # ============================================
    
    def create_existing_assets(self):
        """Create asset nodes for buildings that already have installations"""
        logger.info("Creating existing asset nodes...")
        
        with self.driver.session() as session:
            # Existing solar systems
            result = session.run("""
                MATCH (b:Building {has_solar: true})
                CREATE (s:SolarSystem {
                    system_id: 'SOLAR_EXISTING_' + toString(b.ogc_fid),
                    building_id: b.ogc_fid,
                    status: 'existing',
                    installed_capacity_kwp: b.solar_capacity_kwp,
                    installation_year: 2023,
                    degradation_factor: 0.98,
                    orientation_efficiency: 
                        CASE b.building_orientation_cardinal
                            WHEN 'S' THEN 1.0
                            WHEN 'SE' THEN 0.95
                            WHEN 'SW' THEN 0.95
                            ELSE 0.8
                        END
                })
                CREATE (b)-[:HAS_INSTALLED {
                    install_date: date('2023-01-01')
                }]->(s)
                RETURN count(*) as count
            """)
            solar_count = result.single()['count']
            
            # Existing battery systems
            result = session.run("""
                MATCH (b:Building {has_battery: true})
                CREATE (bat:BatterySystem {
                    system_id: 'BATTERY_EXISTING_' + toString(b.ogc_fid),
                    building_id: b.ogc_fid,
                    status: 'existing',
                    installed_capacity_kwh: 
                        CASE 
                            WHEN b.area < 150 THEN 5.0
                            WHEN b.area < 300 THEN 10.0
                            ELSE 15.0
                        END,
                    power_rating_kw: 
                        CASE 
                            WHEN b.area < 150 THEN 2.5
                            WHEN b.area < 300 THEN 5.0
                            ELSE 7.5
                        END,
                    round_trip_efficiency: 0.9
                })
                CREATE (b)-[:HAS_INSTALLED {
                    install_date: date('2023-01-01')
                }]->(bat)
                RETURN count(*) as count
            """)
            battery_count = result.single()['count']
            
            # Existing heat pump systems
            result = session.run("""
                MATCH (b:Building {has_heat_pump: true})
                CREATE (hp:HeatPumpSystem {
                    system_id: 'HP_EXISTING_' + toString(b.ogc_fid),
                    building_id: b.ogc_fid,
                    status: 'existing',
                    expected_cop: b.expected_cop,
                    heating_capacity_kw: b.area * 0.05,
                    installation_year: 2022
                })
                CREATE (b)-[:HAS_INSTALLED {
                    install_date: date('2022-01-01')
                }]->(hp)
                RETURN count(*) as count
            """)
            hp_count = result.single()['count']
            
            self.stats['nodes_created']['ExistingSolar'] = solar_count
            self.stats['nodes_created']['ExistingBattery'] = battery_count  
            self.stats['nodes_created']['ExistingHeatPump'] = hp_count
            self.stats['relationships_created']['HAS_INSTALLED'] = solar_count + battery_count + hp_count
            
        logger.info(f"Created existing assets: {solar_count} solar, {battery_count} battery, {hp_count} heat pumps")
    
    # ============================================
    # STEP 5: LOAD TEMPORAL ENERGY DATA (UNCHANGED)
    # ============================================
    
    def load_temporal_data(self, data_path: str):
        """Load time-series energy profiles from parquet"""
        logger.info("Loading temporal energy data...")
        
        # Load energy profiles from parquet
        profiles = pd.read_parquet(f"{data_path}/energy_profiles.parquet")
        
        # Get unique timestamps
        timestamps = profiles['timestamp'].unique()
        
        with self.driver.session() as session:
            # Create TimeSlot nodes
            for i, ts in enumerate(timestamps):
                dt = pd.to_datetime(ts)
                session.run("""
                    CREATE (t:TimeSlot {
                        slot_id: $slot_id,
                        timestamp: datetime($timestamp),
                        hour_of_day: $hour,
                        day_of_week: $dow,
                        is_weekend: $weekend,
                        season: $season,
                        time_of_day: $tod
                    })
                """,
                slot_id=f"TS_{i}",
                timestamp=dt.isoformat(),
                hour=dt.hour,
                dow=dt.dayofweek,
                weekend=dt.dayofweek >= 5,
                season=self._get_season(dt),
                tod=self._get_time_of_day(dt.hour)
                )
            
            self.stats['nodes_created']['TimeSlot'] = len(timestamps)
            
            # Create EnergyState nodes in batches
            logger.info("Creating energy states (this may take a moment)...")
            
            batch_size = 1000
            state_count = 0
            
            for building_id in profiles['building_id'].unique():
                building_data = profiles[profiles['building_id'] == building_id]
                
                states = []
                for idx, row in building_data.iterrows():
                    dt = pd.to_datetime(row['timestamp'])
                    slot_id = f"TS_{list(timestamps).index(row['timestamp'])}"
                    
                    # Calculate net position
                    net_demand = row['electricity_demand_kw'] - row['solar_generation_kw'] + \
                                row['battery_discharge_kw'] - row['battery_charge_kw']
                    
                    states.append({
                        'state_id': f"ES_{building_id}_{slot_id}",
                        'building_id': int(building_id),
                        'timeslot_id': slot_id,
                        'electricity_demand_kw': float(row['electricity_demand_kw']),
                        'heating_demand_kw': float(row['heating_demand_kw']),
                        'cooling_demand_kw': float(row['cooling_demand_kw']),
                        'solar_generation_kw': float(row['solar_generation_kw']),
                        'battery_soc_kwh': float(row['battery_soc_kwh']),
                        'battery_charge_kw': float(row['battery_charge_kw']),
                        'battery_discharge_kw': float(row['battery_discharge_kw']),
                        'net_demand_kw': float(net_demand),
                        'is_surplus': net_demand < 0,
                        'export_potential_kw': max(0, -net_demand),
                        'import_need_kw': max(0, net_demand)
                    })
                    
                    if len(states) >= batch_size:
                        self._create_energy_states_batch(session, states)
                        state_count += len(states)
                        states = []
                
                if states:
                    self._create_energy_states_batch(session, states)
                    state_count += len(states)
            
            self.stats['nodes_created']['EnergyState'] = state_count
            
            # Create relationships
            logger.info("Creating temporal relationships...")
            
            # Building -> EnergyState
            result = session.run("""
                MATCH (b:Building), (e:EnergyState {building_id: b.ogc_fid})
                CREATE (b)-[:HAS_STATE_AT]->(e)
                RETURN count(*) as count
            """)
            self.stats['relationships_created']['HAS_STATE_AT'] = result.single()['count']
            
            # EnergyState -> TimeSlot
            result = session.run("""
                MATCH (e:EnergyState), (t:TimeSlot {slot_id: e.timeslot_id})
                CREATE (e)-[:DURING]->(t)
                RETURN count(*) as count
            """)
            self.stats['relationships_created']['DURING'] = result.single()['count']
            
        logger.info(f"Created {state_count} energy states with temporal relationships")
    
    def _create_energy_states_batch(self, session, states):
        """Create energy state nodes in batch"""
        session.run("""
            UNWIND $states as state
            CREATE (e:EnergyState {
                state_id: state.state_id,
                building_id: state.building_id,
                timeslot_id: state.timeslot_id,
                electricity_demand_kw: state.electricity_demand_kw,
                heating_demand_kw: state.heating_demand_kw,
                cooling_demand_kw: state.cooling_demand_kw,
                solar_generation_kw: state.solar_generation_kw,
                battery_soc_kwh: state.battery_soc_kwh,
                battery_charge_kw: state.battery_charge_kw,
                battery_discharge_kw: state.battery_discharge_kw,
                net_demand_kw: state.net_demand_kw,
                is_surplus: state.is_surplus,
                export_potential_kw: state.export_potential_kw,
                import_need_kw: state.import_need_kw
            })
        """, states=states)
    
    def _get_season(self, dt):
        """Get season from datetime"""
        month = dt.month
        if month in [12, 1, 2]:
            return 'winter'
        elif month in [3, 4, 5]:
            return 'spring'
        elif month in [6, 7, 8]:
            return 'summer'
        else:
            return 'fall'
    
    def _get_time_of_day(self, hour):
        """Categorize time of day"""
        if 6 <= hour < 12:
            return 'morning'
        elif 12 <= hour < 18:
            return 'afternoon'
        elif 18 <= hour < 22:
            return 'evening'
        else:
            return 'night'
    
    # ============================================
    # STEP 6: IDENTIFY ASSET OPPORTUNITIES
    # ============================================
    
    def identify_asset_opportunities(self):
        """Create nodes for potential solar, battery, and electrification"""
        logger.info("Identifying asset deployment opportunities...")
        
        with self.driver.session() as session:
            # Solar opportunities (for buildings without solar)
            result = session.run("""
                MATCH (b:Building)
                WHERE b.solar_potential IN ['high', 'medium']
                  AND b.has_solar = false
                CREATE (s:SolarSystem {
                    system_id: 'SOLAR_POTENTIAL_' + toString(b.ogc_fid),
                    building_id: b.ogc_fid,
                    status: 'potential',
                    potential_capacity_kwp: b.solar_capacity_kwp,
                    recommended_capacity_kwp: 
                        CASE 
                            WHEN b.area < 150 THEN 4.0
                            WHEN b.area < 300 THEN 6.0
                            ELSE 10.0
                        END,
                    orientation_efficiency: 
                        CASE b.building_orientation_cardinal
                            WHEN 'S' THEN 1.0
                            WHEN 'SE' THEN 0.95
                            WHEN 'SW' THEN 0.95
                            WHEN 'E' THEN 0.85
                            WHEN 'W' THEN 0.85
                            ELSE 0.7
                        END,
                    payback_years: 7.5
                })
                CREATE (b)-[:CAN_INSTALL {
                    feasibility_score: 
                        CASE b.solar_potential
                            WHEN 'high' THEN 0.9
                            WHEN 'medium' THEN 0.7
                            ELSE 0.5
                        END,
                    priority: 'medium'
                }]->(s)
                RETURN count(*) as count
            """)
            
            solar_count = result.single()['count']
            self.stats['nodes_created']['SolarPotential'] = solar_count
            
            # Battery opportunities
            result = session.run("""
                MATCH (b:Building)
                WHERE (b.solar_potential IN ['high', 'medium'] OR b.building_function = 'non_residential')
                  AND b.has_battery = false
                WITH b, 
                     CASE 
                         WHEN b.area < 150 THEN 5.0
                         WHEN b.area < 300 THEN 10.0
                         ELSE 20.0
                     END as battery_size
                CREATE (bat:BatterySystem {
                    system_id: 'BATTERY_POTENTIAL_' + toString(b.ogc_fid),
                    building_id: b.ogc_fid,
                    status: 'potential',
                    recommended_capacity_kwh: battery_size,
                    power_rating_kw: battery_size / 4,
                    round_trip_efficiency: 0.9,
                    estimated_cycles_per_year: 250
                })
                CREATE (b)-[:CAN_INSTALL {
                    feasibility_score: 
                        CASE 
                            WHEN b.has_solar OR b.solar_potential IN ['high', 'medium'] THEN 0.8
                            ELSE 0.6
                        END,
                    requires_solar: NOT b.has_solar,
                    priority: 'low'
                }]->(bat)
                RETURN count(*) as count
            """)
            
            battery_count = result.single()['count']
            self.stats['nodes_created']['BatteryPotential'] = battery_count
            
            # Electrification opportunities (heat pumps)
            result = session.run("""
                MATCH (b:Building)
                WHERE b.energy_label_simple IN ['D', 'E', 'F', 'G']
                  AND b.has_heat_pump = false
                CREATE (hp:HeatPumpSystem {
                    system_id: 'HP_POTENTIAL_' + toString(b.ogc_fid),
                    building_id: b.ogc_fid,
                    status: 'potential',
                    expected_cop: b.expected_cop,
                    heating_capacity_kw: b.area * 0.05,
                    upgrade_required: b.electrification_feasibility = 'upgrade_needed',
                    estimated_annual_savings_euro: 
                        CASE b.energy_label_simple
                            WHEN 'G' THEN 2000
                            WHEN 'F' THEN 1500
                            WHEN 'E' THEN 1000
                            ELSE 500
                        END
                })
                CREATE (b)-[:SHOULD_ELECTRIFY {
                    priority: 
                        CASE b.energy_label_simple
                            WHEN 'G' THEN 1
                            WHEN 'F' THEN 2
                            WHEN 'E' THEN 3
                            WHEN 'D' THEN 4
                            ELSE 5
                        END,
                    expected_cop: b.expected_cop,
                    requires_insulation_upgrade: b.energy_label_simple IN ['F', 'G']
                }]->(hp)
                RETURN count(*) as count
            """)
            
            hp_count = result.single()['count']
            self.stats['nodes_created']['HeatPumpPotential'] = hp_count
            
        logger.info(f"Identified opportunities: {solar_count} solar, {battery_count} battery, {hp_count} heat pumps")
    
    # ============================================
    # STEP 7: CALCULATE BASELINE METRICS
    # ============================================
    
    def calculate_baseline_metrics(self):
        """Calculate current state metrics for comparison after GNN optimization"""
        logger.info("Calculating baseline metrics...")
        
        with self.driver.session() as session:
            # Building-level statistics
            session.run("""
                MATCH (b:Building)-[:HAS_STATE_AT]->(e:EnergyState)
                WITH b, 
                    max(e.electricity_demand_kw) as peak_demand,
                    avg(e.electricity_demand_kw) as avg_demand,
                    stdev(e.electricity_demand_kw) as demand_std,
                    max(e.solar_generation_kw) as peak_solar,
                    avg(e.net_demand_kw) as avg_net_demand
                SET b.peak_demand_kw = peak_demand,
                    b.avg_demand_kw = avg_demand,
                    b.load_factor = avg_demand / CASE WHEN peak_demand > 0 THEN peak_demand ELSE 1 END,
                    b.demand_variability = demand_std / CASE WHEN avg_demand > 0 THEN avg_demand ELSE 1 END,
                    b.peak_solar_kw = peak_solar,
                    b.avg_net_demand_kw = avg_net_demand,
                    b.self_consumption_ratio = 
                        CASE WHEN b.has_solar 
                        THEN (avg_demand - avg_net_demand) / CASE WHEN avg_demand > 0 THEN avg_demand ELSE 1 END
                        ELSE 0 END
            """)
            
            # LV Cable Group baseline statistics
            session.run("""
                MATCH (g:CableGroup {voltage_level: 'LV'})<-[:CONNECTED_TO]-(b:Building)
                WITH g, 
                    count(b) as building_count,
                    sum(b.peak_demand_kw) as total_peak_demand,
                    avg(b.peak_demand_kw) as avg_building_peak,
                    sum(b.avg_demand_kw) as total_avg_demand,
                    sum(CASE WHEN b.has_solar THEN 1 ELSE 0 END) as solar_count,
                    sum(CASE WHEN b.has_battery THEN 1 ELSE 0 END) as battery_count,
                    sum(CASE WHEN b.has_heat_pump THEN 1 ELSE 0 END) as hp_count,
                    collect(DISTINCT b.building_function) as building_types
                SET g.baseline_building_count = building_count,
                    g.baseline_peak_kw = total_peak_demand,
                    g.baseline_avg_demand_kw = total_avg_demand,
                    g.baseline_load_factor = total_avg_demand / CASE WHEN total_peak_demand > 0 THEN total_peak_demand ELSE 1 END,
                    g.baseline_solar_penetration = toFloat(solar_count) / building_count,
                    g.baseline_battery_penetration = toFloat(battery_count) / building_count,
                    g.baseline_hp_penetration = toFloat(hp_count) / building_count,
                    g.baseline_diversity = size(building_types)
            """)
            
            # Transformer baseline
            session.run("""
                MATCH (t:Transformer)<-[:CONNECTS_TO]-(g:CableGroup {voltage_level: 'LV'})
                WITH t, 
                    count(g) as lv_count,
                    sum(g.baseline_peak_kw) as total_peak,
                    sum(g.baseline_building_count) as total_buildings
                SET t.baseline_lv_count = lv_count,
                    t.baseline_peak_kw = total_peak,
                    t.baseline_building_count = total_buildings
            """)
            
            # System-wide baseline
            result = session.run("""
                MATCH (b:Building)
                WITH count(b) as total_buildings,
                    sum(b.peak_demand_kw) as system_peak,
                    avg(b.load_factor) as avg_load_factor,
                    sum(CASE WHEN b.has_solar THEN 1 ELSE 0 END) as solar_buildings,
                    sum(CASE WHEN b.has_battery THEN 1 ELSE 0 END) as battery_buildings,
                    sum(CASE WHEN b.has_heat_pump THEN 1 ELSE 0 END) as hp_buildings
                CREATE (s:SystemBaseline {
                    id: 'BASELINE_' + toString(datetime()),
                    timestamp: datetime(),
                    total_buildings: total_buildings,
                    system_peak_kw: system_peak,
                    avg_load_factor: avg_load_factor,
                    solar_penetration: toFloat(solar_buildings) / total_buildings,
                    battery_penetration: toFloat(battery_buildings) / total_buildings,
                    hp_penetration: toFloat(hp_buildings) / total_buildings,
                    description: 'Pre-GNN optimization baseline'
                })
                RETURN total_buildings, system_peak, avg_load_factor
            """)
            
            baseline = result.single()
            if baseline:
                total_buildings = baseline['total_buildings'] or 0
                system_peak = baseline['system_peak'] or 0
                avg_load_factor = baseline['avg_load_factor'] or 0
                
                logger.info(f"Baseline: {total_buildings} buildings, "
                        f"{system_peak:.0f} kW peak, "
                        f"{avg_load_factor:.3f} load factor")
            
    # ============================================
    # STEP 8: ADD METADATA
    # ============================================
    
    def add_metadata(self):
        """Add metadata about the KG creation"""
        logger.info("Adding metadata...")
        
        with self.driver.session() as session:
            metadata = {
                'creation_timestamp': datetime.now().isoformat(),
                'total_nodes': sum(self.stats['nodes_created'].values()),
                'total_relationships': sum(self.stats['relationships_created'].values()),
                'node_types': self.stats['nodes_created'],
                'relationship_types': self.stats['relationships_created']
            }
            
            session.run("""
                CREATE (m:Metadata {
                    id: 'PRE_GNN_METADATA',
                    created_at: datetime($timestamp),
                    total_nodes: $nodes,
                    total_relationships: $rels,
                    node_breakdown: $node_types,
                    relationship_breakdown: $rel_types,
                    data_source: 'PostgreSQL',
                    version: '2.0',
                    stage: 'pre_gnn',
                    description: 'Knowledge graph with rich grid infrastructure before GNN optimization'
                })
            """,
            timestamp=metadata['creation_timestamp'],
            nodes=metadata['total_nodes'],
            rels=metadata['total_relationships'],
            node_types=json.dumps(metadata['node_types']),
            rel_types=json.dumps(metadata['relationship_types'])
            )
            
        logger.info("Metadata added")
    
    # ============================================
    # MAIN EXECUTION METHOD
    # ============================================
    
    def build_complete_graph(self, data_path: str, clear_first: bool = True):
        """Build complete knowledge graph from PostgreSQL database (Pre-GNN version)"""
        
        start_time = datetime.now()
        logger.info("="*50)
        logger.info("Starting Knowledge Graph Construction (Pre-GNN)")
        logger.info("="*50)
        
        try:
            # Clear database if requested
            if clear_first:
                self.clear_database()
            
            # Build graph step by step
            self.create_schema()
            self.load_grid_infrastructure()  # From PostgreSQL
            self.load_buildings()  # From PostgreSQL
            self.create_existing_assets()
            self.load_temporal_data(data_path)  # Still from parquet
            self.identify_asset_opportunities()
            self.calculate_baseline_metrics()
            self.add_metadata()
            
            # Calculate total time
            end_time = datetime.now()
            total_time = (end_time - start_time).total_seconds()
            
            # Print summary
            logger.info("="*50)
            logger.info("Pre-GNN Knowledge Graph Construction Complete!")
            logger.info("="*50)
            logger.info(f"Total processing time: {total_time:.2f} seconds")
            logger.info("\nNodes created:")
            for node_type, count in self.stats['nodes_created'].items():
                logger.info(f"  {node_type}: {count:,}")
            logger.info(f"  TOTAL: {sum(self.stats['nodes_created'].values()):,}")
            
            logger.info("\nRelationships created:")
            for rel_type, count in self.stats['relationships_created'].items():
                logger.info(f"  {rel_type}: {count:,}")
            logger.info(f"  TOTAL: {sum(self.stats['relationships_created'].values()):,}")
            
            logger.info("\n⚠️ Note: Complementarity analysis and clustering will be added after GNN processing")
            
            return self.stats
            
        except Exception as e:
            logger.error(f"Error building knowledge graph: {str(e)}")
            raise

# ============================================
# MAIN EXECUTION
# ============================================

if __name__ == "__main__":
    # Configuration
    NEO4J_URI = "bolt://localhost:7687"
    NEO4J_USER = "neo4j"
    NEO4J_PASSWORD = "aminasad"  # Change this to your password
    
    # PostgreSQL configuration
    PG_HOST = "localhost"
    PG_DATABASE = "research"  # Your database name
    PG_USER = "aminj"
    PG_PASSWORD = "Aminej@geodan!"  # Change this

    DATA_PATH = "mimic_data"  # For parquet files
    
    # Create builder and construct graph
    builder = EnergyKnowledgeGraphBuilder(
        NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD,
        PG_HOST, PG_DATABASE, PG_USER, PG_PASSWORD
    )
    
    try:
        # Build complete knowledge graph
        stats = builder.build_complete_graph(DATA_PATH, clear_first=True)
        
        # Run validation queries
        with builder.driver.session() as session:
            # Check grid infrastructure
            result = session.run("""
                MATCH (n:GridComponent)
                RETURN n.component_type as type, count(*) as count
                ORDER BY count DESC
            """)
            
            print("\n" + "="*50)
            print("Grid Infrastructure Summary:")
            print("="*50)
            for record in result:
                print(f"{record['type']}: {record['count']} components")
            
            # Check LV groups with buildings
            result = session.run("""
                MATCH (g:CableGroup {voltage_level: 'LV'})
                OPTIONAL MATCH (g)<-[:CONNECTED_TO]-(b:Building)
                WITH g, count(b) as buildings
                RETURN g.group_id as group_id,
                       buildings,
                       g.baseline_peak_kw as peak_kw,
                       g.baseline_solar_penetration as solar_pen
                ORDER BY buildings DESC
                LIMIT 10
            """)
            
            print("\n" + "="*50)
            print("Top 10 LV Groups by Building Count:")
            print("="*50)
            for record in result:
                print(f"{record['group_id']}: {record['buildings']} buildings, "
                      f"Peak: {record['peak_kw']:.0f} kW, "
                      f"Solar: {record['solar_pen']:.1%}" if record['peak_kw'] else "No metrics yet")
            
            # Check building connection quality
            result = session.run("""
                MATCH (b:Building)
                RETURN b.connection_type as type, 
                       count(*) as count,
                       avg(b.connection_distance_m) as avg_distance
                ORDER BY count DESC
            """)
            
            print("\n" + "="*50)
            print("Building Connection Quality:")
            print("="*50)
            for record in result:
                print(f"{record['type']}: {record['count']} buildings, "
                      f"Avg distance: {record['avg_distance']:.1f}m")
            
            # Check opportunities
            result = session.run("""
                OPTIONAL MATCH (s:SolarSystem {status: 'potential'})
                WITH count(s) as solar_opp
                OPTIONAL MATCH (b:BatterySystem {status: 'potential'})
                WITH solar_opp, count(b) as battery_opp
                OPTIONAL MATCH (h:HeatPumpSystem {status: 'potential'})
                RETURN solar_opp, battery_opp, count(h) as hp_opp
            """)
            
            opp = result.single()
            print("\n" + "="*50)
            print("Deployment Opportunities Identified:")
            print("="*50)
            if opp:
                print(f"Solar: {opp['solar_opp']} buildings")
                print(f"Battery: {opp['battery_opp']} buildings")
                print(f"Heat Pump: {opp['hp_opp']} buildings")
        
    except Exception as e:
        print(f"Error during execution: {str(e)}")
        import traceback
        traceback.print_exc()
    finally:
        builder.close()
    
    print("\n✅ Pre-GNN Knowledge Graph construction complete!")
    print("Ready for GNN processing to discover complementarity and optimal clustering")
    print("You can explore the graph in Neo4j Browser at http://localhost:7474")
    
    """
    # After GNN runs, you'll add:
    1. COMPLEMENTS relationships (discovered by GNN)
    2. EnergyCluster nodes (optimized groupings)
    3. TRADES_ENERGY_WITH relationships
    4. DeploymentScenario nodes (GNN recommendations)
    5. Performance metrics comparing to baseline
    """

2025-08-17 15:53:38,512 - INFO - Connected to Neo4j at bolt://localhost:7687
2025-08-17 15:53:38,513 - INFO - Connected to PostgreSQL database research
2025-08-17 15:53:38,513 - INFO - ==================================================
2025-08-17 15:53:38,513 - INFO - Starting Knowledge Graph Construction (Pre-GNN)
2025-08-17 15:53:38,514 - INFO - ==================================================
2025-08-17 15:53:40,611 - INFO - Neo4j database cleared
2025-08-17 15:53:40,612 - INFO - Creating schema constraints and indexes...
2025-08-17 15:53:40,615 - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:Substation) REQUIRE (e.station_id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_59ea450c FOR (e:Substation) REQUIRE (e.station_id) IS UNIQUE` already exists.} for query: 'CREATE CONSTRAINT IF NOT EXISTS FOR (s:Subst


Grid Infrastructure Summary:
lv_cabinet: 316 components
cable_group: 209 components
transformer: 49 components
substation: 2 components

Top 10 LV Groups by Building Count:
No metrics yet
No metrics yet
No metrics yet
No metrics yet
No metrics yet
No metrics yet
No metrics yet
No metrics yet
No metrics yet
No metrics yet

Building Connection Quality:
BY_DISTANCE: 1440 buildings, Avg distance: 13.9m
ENDED: 67 buildings, Avg distance: 13.8m
CROSSED: 9 buildings, Avg distance: 15.9m
TOO_FAR: 1 buildings, Avg distance: 215.5m


2025-08-17 15:54:42,760 - INFO - Database connections closed



Deployment Opportunities Identified:
Solar: 926 buildings
Battery: 1463 buildings
Heat Pump: 1079 buildings

✅ Pre-GNN Knowledge Graph construction complete!
Ready for GNN processing to discover complementarity and optimal clustering
You can explore the graph in Neo4j Browser at http://localhost:7474


In [6]:
"""
Adjacency Module for Energy KG - Part 2
Updates existing Knowledge Graph with adjacency relationships
Uses PostgreSQL database (same as Part 1) instead of CSV
"""

import pandas as pd
import numpy as np
from neo4j import GraphDatabase
import psycopg2
from psycopg2.extras import RealDictCursor
from datetime import datetime
import logging
from typing import Dict, List, Tuple, Optional
from decimal import Decimal

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class AdjacencyUpdater:
    """Add adjacency relationships to existing Knowledge Graph using PostgreSQL data"""
    
    def __init__(self, neo4j_uri: str, neo4j_user: str, neo4j_password: str,
                 pg_host: str, pg_database: str, pg_user: str, pg_password: str):
        """Initialize Neo4j and PostgreSQL connections (same as Part 1)"""
        # Neo4j connection
        self.driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))
        
        # PostgreSQL connection (same as Part 1)
        self.pg_conn = psycopg2.connect(
            host=pg_host,
            database=pg_database,
            user=pg_user,
            password=pg_password,
            port=5433
        )
        
        self.stats = {
            'relationships_created': 0,
            'clusters_created': 0,
            'nodes_updated': 0,
            'validation_results': {}
        }
        logger.info(f"Connected to Neo4j and PostgreSQL for adjacency update")
    
    def close(self):
        """Close database connections"""
        self.driver.close()
        self.pg_conn.close()
        logger.info("Connections closed")
    
    def convert_decimals(self, record):
        """Convert Decimal types to float for Neo4j compatibility (from Part 1)"""
        if isinstance(record, dict):
            return {k: float(v) if isinstance(v, Decimal) else v for k, v in record.items()}
        elif isinstance(record, list):
            return [self.convert_decimals(item) for item in record]
        else:
            return float(record) if isinstance(record, Decimal) else record
    
    def safe_float(self, value, default=0.0) -> float:
        """Safely convert value to float, handling None and other edge cases"""
        if value is None:
            return default
        try:
            return float(value)
        except (TypeError, ValueError):
            return default
    
    def safe_int(self, value, default=0) -> int:
        """Safely convert value to int, handling None and other edge cases"""
        if value is None:
            return default
        try:
            return int(value)
        except (TypeError, ValueError):
            return default
    
    def check_kg_status(self) -> Dict:
        """Check if KG exists and has required data"""
        with self.driver.session() as session:
            result = session.run("""
                MATCH (b:Building)
                RETURN 
                    count(b) as building_count,
                    count(b.north_shared_length) as has_shared_wall_data,
                    exists((b)-[:ADJACENT_TO]-()) as has_adjacencies
                LIMIT 1
            """)
            
            status = result.single()
            
            return {
                'buildings_exist': status['building_count'] > 0,
                'has_shared_walls': status['has_shared_wall_data'] > 0,
                'adjacencies_exist': status['has_adjacencies'],
                'building_count': status['building_count']
            }
    
    def update_shared_wall_data_from_postgres(self):
        """Update existing Neo4j buildings with shared wall data from PostgreSQL"""
        logger.info("Updating buildings with shared wall data from PostgreSQL...")
        
        with self.pg_conn.cursor(cursor_factory=RealDictCursor) as cursor:
            # Query buildings with shared wall data from PostgreSQL
            cursor.execute("""
                SELECT 
                    b.ogc_fid,
                    -- Shared wall lengths (handle NULLs with COALESCE)
                    COALESCE(b.north_shared_length, 0) as north_shared_length,
                    COALESCE(b.south_shared_length, 0) as south_shared_length,
                    COALESCE(b.east_shared_length, 0) as east_shared_length,
                    COALESCE(b.west_shared_length, 0) as west_shared_length,
                    -- Facade lengths (handle NULLs with COALESCE)
                    COALESCE(b.north_facade_length, 10) as north_facade_length,
                    COALESCE(b.south_facade_length, 10) as south_facade_length,
                    COALESCE(b.east_facade_length, 10) as east_facade_length,
                    COALESCE(b.west_facade_length, 10) as west_facade_length,
                    -- Calculate derived fields
                    (CASE WHEN b.north_shared_length > 0 THEN 1 ELSE 0 END +
                     CASE WHEN b.south_shared_length > 0 THEN 1 ELSE 0 END +
                     CASE WHEN b.east_shared_length > 0 THEN 1 ELSE 0 END +
                     CASE WHEN b.west_shared_length > 0 THEN 1 ELSE 0 END) as num_shared_walls,
                    (COALESCE(b.north_shared_length, 0) + 
                     COALESCE(b.south_shared_length, 0) + 
                     COALESCE(b.east_shared_length, 0) + 
                     COALESCE(b.west_shared_length, 0)) as total_shared_length,
                    -- Adjacency type classification
                    CASE 
                        WHEN (CASE WHEN b.north_shared_length > 0 THEN 1 ELSE 0 END +
                              CASE WHEN b.south_shared_length > 0 THEN 1 ELSE 0 END +
                              CASE WHEN b.east_shared_length > 0 THEN 1 ELSE 0 END +
                              CASE WHEN b.west_shared_length > 0 THEN 1 ELSE 0 END) = 0 
                        THEN 'ISOLATED'
                        
                        WHEN (b.north_shared_length > 0 AND b.south_shared_length > 0) OR
                             (b.east_shared_length > 0 AND b.west_shared_length > 0)
                        THEN 'MIDDLE_ROW'
                        
                        WHEN (CASE WHEN b.north_shared_length > 0 THEN 1 ELSE 0 END +
                              CASE WHEN b.south_shared_length > 0 THEN 1 ELSE 0 END +
                              CASE WHEN b.east_shared_length > 0 THEN 1 ELSE 0 END +
                              CASE WHEN b.west_shared_length > 0 THEN 1 ELSE 0 END) = 2
                        THEN 'CORNER'
                        
                        WHEN (CASE WHEN b.north_shared_length > 0 THEN 1 ELSE 0 END +
                              CASE WHEN b.south_shared_length > 0 THEN 1 ELSE 0 END +
                              CASE WHEN b.east_shared_length > 0 THEN 1 ELSE 0 END +
                              CASE WHEN b.west_shared_length > 0 THEN 1 ELSE 0 END) = 1
                        THEN 'END_ROW'
                        
                        WHEN (CASE WHEN b.north_shared_length > 0 THEN 1 ELSE 0 END +
                              CASE WHEN b.south_shared_length > 0 THEN 1 ELSE 0 END +
                              CASE WHEN b.east_shared_length > 0 THEN 1 ELSE 0 END +
                              CASE WHEN b.west_shared_length > 0 THEN 1 ELSE 0 END) >= 3
                        THEN 'COURTYARD'
                        
                        ELSE 'UNKNOWN'
                    END as adjacency_type,
                    -- Additional context
                    b.woningtype,
                    b.area
                FROM amin.buildings_1_deducted b
                JOIN amin_grid.tlip_building_connections bc 
                    ON b.ogc_fid = bc.building_id
                WHERE b.area > 10 
                    AND b.pand_geom IS NOT NULL
                    AND bc.connected_group_id IS NOT NULL
            """)
            
            buildings = [self.convert_decimals(row) for row in cursor.fetchall()]
        
        # Update Neo4j buildings with shared wall data
        with self.driver.session() as session:
            updated = 0
            batch_size = 500
            batch = []
            
            for building in buildings:
                # Use safe conversion functions to handle None values
                batch.append({
                    'ogc_fid': self.safe_int(building['ogc_fid']),
                    'north_shared': self.safe_float(building['north_shared_length'], 0),
                    'south_shared': self.safe_float(building['south_shared_length'], 0),
                    'east_shared': self.safe_float(building['east_shared_length'], 0),
                    'west_shared': self.safe_float(building['west_shared_length'], 0),
                    'north_facade': self.safe_float(building['north_facade_length'], 10),
                    'south_facade': self.safe_float(building['south_facade_length'], 10),
                    'east_facade': self.safe_float(building['east_facade_length'], 10),
                    'west_facade': self.safe_float(building['west_facade_length'], 10),
                    'num_shared': self.safe_int(building['num_shared_walls'], 0),
                    'total_shared': self.safe_float(building['total_shared_length'], 0),
                    'adj_type': building.get('adjacency_type', 'UNKNOWN') or 'UNKNOWN',
                    'housing_type': building.get('woningtype', 'unknown') or 'unknown'
                })
                
                if len(batch) >= batch_size:
                    result = session.run("""
                        UNWIND $batch as building
                        MATCH (b:Building {ogc_fid: building.ogc_fid})
                        SET b.north_shared_length = building.north_shared,
                            b.south_shared_length = building.south_shared,
                            b.east_shared_length = building.east_shared,
                            b.west_shared_length = building.west_shared,
                            b.north_facade_length = building.north_facade,
                            b.south_facade_length = building.south_facade,
                            b.east_facade_length = building.east_facade,
                            b.west_facade_length = building.west_facade,
                            b.num_shared_walls = building.num_shared,
                            b.total_shared_length = building.total_shared,
                            b.adjacency_type = building.adj_type,
                            b.woningtype = building.housing_type
                        RETURN count(b) as updated_count
                    """, batch=batch)
                    
                    single_result = result.single()
                    if single_result:
                        updated += single_result['updated_count']
                    batch = []
            
            # Process remaining batch
            if batch:
                result = session.run("""
                    UNWIND $batch as building
                    MATCH (b:Building {ogc_fid: building.ogc_fid})
                    SET b.north_shared_length = building.north_shared,
                        b.south_shared_length = building.south_shared,
                        b.east_shared_length = building.east_shared,
                        b.west_shared_length = building.west_shared,
                        b.north_facade_length = building.north_facade,
                        b.south_facade_length = building.south_facade,
                        b.east_facade_length = building.east_facade,
                        b.west_facade_length = building.west_facade,
                        b.num_shared_walls = building.num_shared,
                        b.total_shared_length = building.total_shared,
                        b.adjacency_type = building.adj_type,
                        b.woningtype = building.housing_type
                    RETURN count(b) as updated_count
                """, batch=batch)
                
                single_result = result.single()
                if single_result:
                    updated += single_result['updated_count']
            
            self.stats['nodes_updated'] = updated
            logger.info(f"Updated {updated} buildings with shared wall data from PostgreSQL")
    
    def create_adjacency_relationships(self):
        """Find and create ADJACENT_TO relationships between buildings"""
        logger.info("Creating adjacency relationships...")
        
        with self.driver.session() as session:
            # Clear any existing adjacency relationships first
            session.run("MATCH ()-[r:ADJACENT_TO]-() DELETE r")
            
            # Create relationships using spatial proximity and shared walls
            result = session.run("""
                // Find all buildings with shared walls
                MATCH (b1:Building)
                WHERE b1.num_shared_walls > 0
                
                // Find potential neighbors in same LV group
                MATCH (b2:Building)
                WHERE b1.ogc_fid < b2.ogc_fid  // Process each pair only once
                AND b1.lv_group_id = b2.lv_group_id  // Same LV network
                AND b2.num_shared_walls > 0
                
                // Calculate distance (must be very close for true adjacency)
                WITH b1, b2, 
                    sqrt((b2.x - b1.x)^2 + (b2.y - b1.y)^2) as distance
                WHERE distance < 5  // Reduced to 5m for true adjacency
                
                // Check for matching shared walls
                WITH b1, b2, distance,
                    CASE 
                        // North-South match
                        WHEN b1.north_shared_length > 0 AND b2.south_shared_length > 0 
                            AND b2.y > b1.y AND abs(b2.x - b1.x) < 3
                        THEN {
                            pair: 'north-south',
                            b1_wall: 'north',
                            b2_wall: 'south',
                            b1_length: b1.north_shared_length,
                            b2_length: b2.south_shared_length,
                            match_quality: 
                                CASE 
                                    WHEN b1.north_shared_length < b2.south_shared_length 
                                    THEN b1.north_shared_length / b2.south_shared_length
                                    ELSE b2.south_shared_length / b1.north_shared_length
                                END
                        }
                        // South-North match
                        WHEN b1.south_shared_length > 0 AND b2.north_shared_length > 0
                            AND b2.y < b1.y AND abs(b2.x - b1.x) < 3
                        THEN {
                            pair: 'south-north',
                            b1_wall: 'south',
                            b2_wall: 'north',
                            b1_length: b1.south_shared_length,
                            b2_length: b2.north_shared_length,
                            match_quality: 
                                CASE 
                                    WHEN b1.south_shared_length < b2.north_shared_length 
                                    THEN b1.south_shared_length / b2.north_shared_length
                                    ELSE b2.north_shared_length / b1.south_shared_length
                                END
                        }
                        // East-West match
                        WHEN b1.east_shared_length > 0 AND b2.west_shared_length > 0
                            AND b2.x > b1.x AND abs(b2.y - b1.y) < 3
                        THEN {
                            pair: 'east-west',
                            b1_wall: 'east',
                            b2_wall: 'west',
                            b1_length: b1.east_shared_length,
                            b2_length: b2.west_shared_length,
                            match_quality: 
                                CASE 
                                    WHEN b1.east_shared_length < b2.west_shared_length 
                                    THEN b1.east_shared_length / b2.west_shared_length
                                    ELSE b2.west_shared_length / b1.east_shared_length
                                END
                        }
                        // West-East match
                        WHEN b1.west_shared_length > 0 AND b2.east_shared_length > 0
                            AND b2.x < b1.x AND abs(b2.y - b1.y) < 3
                        THEN {
                            pair: 'west-east',
                            b1_wall: 'west',
                            b2_wall: 'east',
                            b1_length: b1.west_shared_length,
                            b2_length: b2.east_shared_length,
                            match_quality: 
                                CASE 
                                    WHEN b1.west_shared_length < b2.east_shared_length 
                                    THEN b1.west_shared_length / b2.east_shared_length
                                    ELSE b2.east_shared_length / b1.west_shared_length
                                END
                        }
                        ELSE null
                    END as wall_match
                
                WHERE wall_match IS NOT NULL
                AND wall_match.match_quality > 0.5  // Walls must be similar length
                
                // Create bidirectional relationships
                CREATE (b1)-[r1:ADJACENT_TO {
                    wall_pair: wall_match.pair,
                    my_wall: wall_match.b1_wall,
                    their_wall: wall_match.b2_wall,
                    my_shared_length: wall_match.b1_length,
                    their_shared_length: wall_match.b2_length,
                    match_quality: wall_match.match_quality,
                    distance_m: distance,
                    adjacency_strength: (wall_match.b1_length + wall_match.b2_length) / 2,
                    
                    // Energy implications
                    thermal_coupling: true,
                    cable_distance: distance * 0.5,
                    transmission_loss_factor: 0.02 * distance,
                    energy_sharing_viable: distance < 3,
                    
                    // Complementarity based on function
                    function_diversity: CASE 
                        WHEN b1.building_function <> b2.building_function THEN 2.0
                        WHEN b1.residential_type <> b2.residential_type THEN 1.5
                        ELSE 1.0
                    END,
                    
                    // Solar diversity
                    solar_diversity: CASE
                        WHEN b1.has_solar <> b2.has_solar THEN 2.0
                        WHEN b1.solar_potential <> b2.solar_potential THEN 1.5
                        ELSE 1.0
                    END,
                    
                    created_at: datetime()
                }]->(b2)
                
                CREATE (b2)-[r2:ADJACENT_TO {
                    wall_pair: wall_match.b2_wall + '-' + wall_match.b1_wall,
                    my_wall: wall_match.b2_wall,
                    their_wall: wall_match.b1_wall,
                    my_shared_length: wall_match.b2_length,
                    their_shared_length: wall_match.b1_length,
                    match_quality: wall_match.match_quality,
                    distance_m: distance,
                    adjacency_strength: (wall_match.b1_length + wall_match.b2_length) / 2,
                    
                    // Same energy implications
                    thermal_coupling: true,
                    cable_distance: distance * 0.5,
                    transmission_loss_factor: 0.02 * distance,
                    energy_sharing_viable: distance < 3,
                    
                    function_diversity: CASE 
                        WHEN b1.building_function <> b2.building_function THEN 2.0
                        WHEN b1.residential_type <> b2.residential_type THEN 1.5
                        ELSE 1.0
                    END,
                    
                    solar_diversity: CASE
                        WHEN b1.has_solar <> b2.has_solar THEN 2.0
                        WHEN b1.solar_potential <> b2.solar_potential THEN 1.5
                        ELSE 1.0
                    END,
                    
                    created_at: datetime()
                }]->(b1)
                
                RETURN count(DISTINCT r1) as relationships_created
            """)
            
            single_result = result.single()
            count = single_result['relationships_created'] if single_result else 0
            self.stats['relationships_created'] = count * 2  # Both directions
            logger.info(f"Created {count * 2} adjacency relationships ({count} pairs)")
            
            return count * 2
    
    def validate_adjacencies(self) -> Dict:
        """Validate adjacency patterns match expectations based on housing types"""
        logger.info("Validating adjacency patterns...")
        
        with self.driver.session() as session:
            validations = {}
            
            # Validation 1: Row houses (rijtjeswoning) should have 2 neighbors
            result = session.run("""
                MATCH (b:Building)
                WHERE b.woningtype = 'rijtjeswoning'
                OPTIONAL MATCH (b)-[r:ADJACENT_TO]-()
                WITH b, count(DISTINCT r) as neighbor_count
                RETURN 
                    count(b) as total,
                    sum(CASE WHEN neighbor_count = 2 THEN 1 ELSE 0 END) as correct,
                    avg(neighbor_count) as avg_neighbors
            """)
            row = result.single()
            if row and row['total'] > 0:
                validations['row_houses'] = {
                    'total': row['total'],
                    'correct': row['correct'] or 0,
                    'avg_neighbors': row['avg_neighbors'] or 0,
                    'accuracy': (row['correct'] or 0) / row['total']
                }
            
            # Validation 2: Detached (vrijstaand) should have 0 neighbors
            result = session.run("""
                MATCH (b:Building)
                WHERE b.woningtype = 'vrijstaand'
                OPTIONAL MATCH (b)-[r:ADJACENT_TO]-()
                WITH b, count(DISTINCT r) as neighbor_count
                RETURN 
                    count(b) as total,
                    sum(CASE WHEN neighbor_count = 0 THEN 1 ELSE 0 END) as correct,
                    avg(neighbor_count) as avg_neighbors
            """)
            row = result.single()
            if row and row['total'] > 0:
                validations['detached'] = {
                    'total': row['total'],
                    'correct': row['correct'] or 0,
                    'avg_neighbors': row['avg_neighbors'] or 0,
                    'accuracy': (row['correct'] or 0) / row['total']
                }
            
            # Validation 3: Semi-detached (twee_onder_1_kap) should have 1 neighbor
            result = session.run("""
                MATCH (b:Building)
                WHERE b.woningtype = 'twee_onder_1_kap'
                OPTIONAL MATCH (b)-[r:ADJACENT_TO]-()
                WITH b, count(DISTINCT r) as neighbor_count
                RETURN 
                    count(b) as total,
                    sum(CASE WHEN neighbor_count = 1 THEN 1 ELSE 0 END) as correct,
                    avg(neighbor_count) as avg_neighbors
            """)
            row = result.single()
            if row and row['total'] > 0:
                validations['semi_detached'] = {
                    'total': row['total'],
                    'correct': row['correct'] or 0,
                    'avg_neighbors': row['avg_neighbors'] or 0,
                    'accuracy': (row['correct'] or 0) / row['total']
                }
            
            # Validation 4: MIDDLE_ROW adjacency type should match
            result = session.run("""
                MATCH (b:Building {adjacency_type: 'MIDDLE_ROW'})
                OPTIONAL MATCH (b)-[r:ADJACENT_TO]-()
                WITH b, count(DISTINCT r) as neighbor_count
                RETURN 
                    count(b) as total,
                    sum(CASE WHEN neighbor_count = 2 THEN 1 ELSE 0 END) as correct,
                    avg(neighbor_count) as avg_neighbors
            """)
            row = result.single()
            if row and row['total'] > 0:
                validations['middle_row_type'] = {
                    'total': row['total'],
                    'correct': row['correct'] or 0,
                    'avg_neighbors': row['avg_neighbors'] or 0,
                    'accuracy': (row['correct'] or 0) / row['total']
                }
            
            # Validation 5: Reciprocal relationships
            result = session.run("""
                MATCH (b1:Building)-[r:ADJACENT_TO]->(b2:Building)
                WHERE b1.ogc_fid < b2.ogc_fid
                OPTIONAL MATCH (b2)-[r2:ADJACENT_TO]->(b1)
                RETURN 
                    count(r) as total_relationships,
                    count(r2) as reciprocal_relationships
            """)
            row = result.single()
            if row and row['total_relationships'] > 0:
                validations['reciprocal'] = {
                    'total': row['total_relationships'],
                    'reciprocal': row['reciprocal_relationships'] or 0,
                    'accuracy': (row['reciprocal_relationships'] or 0) / row['total_relationships']
                }
            
            self.stats['validation_results'] = validations
            return validations
    
    def create_adjacency_clusters(self):
        """Create natural clusters from adjacency patterns"""
        logger.info("Creating natural adjacency clusters...")
        
        with self.driver.session() as session:
            # Clear existing adjacency clusters
            session.run("MATCH (ac:AdjacencyCluster) DETACH DELETE ac")
            
            total_clusters = 0
            
            # Method 1: Row House Clusters - FIXED to use elementId()
            logger.info("Creating row house clusters...")
            
            result = session.run("""
                // Find unprocessed row houses
                MATCH (start:Building)
                WHERE (start.adjacency_type = 'MIDDLE_ROW' OR start.adjacency_type = 'END_ROW')
                AND NOT EXISTS((start)<-[:IN_ADJACENCY_CLUSTER]-())
                
                // Find connected row of buildings
                MATCH path = (start)-[:ADJACENT_TO*1..20]-(connected:Building)
                WHERE connected.adjacency_type IN ['MIDDLE_ROW', 'END_ROW', 'CORNER']
                
                WITH start, collect(DISTINCT connected) + [start] as row_buildings
                WHERE size(row_buildings) >= 3
                
                // Get LV group for cluster
                WITH row_buildings, row_buildings[0].lv_group_id as lv_group,
                    row_buildings[0].district_name as district,
                    row_buildings[0].ogc_fid as first_building_id
                
                CREATE (ac:AdjacencyCluster {
                    cluster_id: 'ROW_' + lv_group + '_' + toString(first_building_id),
                    cluster_type: 'ROW_HOUSES',
                    member_count: size(row_buildings),
                    lv_group_id: lv_group,
                    district_name: district,
                    created_at: datetime(),
                    pattern: 'LINEAR',
                    thermal_benefit: 'HIGH',
                    cable_savings: 'HIGH',
                    avg_shared_walls: 2.0
                })
                
                WITH ac, row_buildings
                UNWIND row_buildings as building
                CREATE (building)-[:IN_ADJACENCY_CLUSTER {
                    joined_at: datetime()
                }]->(ac)
                
                RETURN count(DISTINCT ac) as clusters_created
            """)
            
            row_result = result.single()
            row_count = row_result['clusters_created'] if row_result else 0
            logger.info(f"Created {row_count} row house clusters")
            
            # Method 2: Corner/Courtyard Clusters
            logger.info("Creating corner/courtyard clusters...")
            
            result = session.run("""
                // Find unprocessed corner/courtyard buildings
                MATCH (center:Building)
                WHERE center.adjacency_type IN ['CORNER', 'COURTYARD']
                AND NOT EXISTS((center)<-[:IN_ADJACENCY_CLUSTER]-())
                
                // Find immediate neighbors
                MATCH (center)-[:ADJACENT_TO]-(neighbor:Building)
                WHERE NOT EXISTS((neighbor)<-[:IN_ADJACENCY_CLUSTER]-())
                
                WITH center, collect(DISTINCT neighbor) as neighbors
                WHERE size(neighbors) >= 2
                
                CREATE (ac:AdjacencyCluster {
                    cluster_id: 'CORNER_' + center.lv_group_id + '_' + toString(center.ogc_fid),
                    cluster_type: CASE 
                        WHEN center.adjacency_type = 'COURTYARD' THEN 'COURTYARD_BLOCK'
                        ELSE 'CORNER_BLOCK'
                    END,
                    member_count: size(neighbors) + 1,
                    lv_group_id: center.lv_group_id,
                    district_name: center.district_name,
                    created_at: datetime(),
                    pattern: CASE 
                        WHEN center.adjacency_type = 'COURTYARD' THEN 'ENCLOSED'
                        ELSE 'L_SHAPE'
                    END,
                    thermal_benefit: 'MEDIUM',
                    cable_savings: 'MEDIUM',
                    avg_shared_walls: 1.5
                })
                
                CREATE (center)-[:IN_ADJACENCY_CLUSTER {
                    role: 'CENTER',
                    joined_at: datetime()
                }]->(ac)
                
                WITH ac, neighbors
                UNWIND neighbors as neighbor
                CREATE (neighbor)-[:IN_ADJACENCY_CLUSTER {
                    role: 'MEMBER',
                    joined_at: datetime()
                }]->(ac)
                
                RETURN count(DISTINCT ac) as created
            """)
            
            corner_result = result.single()
            corner_count = corner_result['created'] if corner_result else 0
            logger.info(f"Created {corner_count} corner/courtyard clusters")
            
            # Method 3: Apartment Complex Clusters - FIXED to use ogc_fid
            logger.info("Creating apartment clusters...")
            
            result = session.run("""
                // Find apartment buildings in same LV group and location
                MATCH (b1:Building)
                WHERE b1.woningtype = 'appartement'
                AND NOT EXISTS((b1)<-[:IN_ADJACENCY_CLUSTER]-())
                
                MATCH (b2:Building)
                WHERE b2.woningtype = 'appartement'
                AND b1.ogc_fid < b2.ogc_fid
                AND b1.lv_group_id = b2.lv_group_id
                AND b1.district_name = b2.district_name
                AND NOT EXISTS((b2)<-[:IN_ADJACENCY_CLUSTER]-())
                AND sqrt((b2.x - b1.x)^2 + (b2.y - b1.y)^2) < 50  // Within 50m
                
                WITH b1.lv_group_id as lv_group, 
                    b1.district_name as district,
                    collect(DISTINCT b1) + collect(DISTINCT b2) as apartments,
                    min(b1.ogc_fid) as min_building_id
                WHERE size(apartments) >= 2
                
                CREATE (ac:AdjacencyCluster {
                    cluster_id: 'APT_' + lv_group + '_' + toString(min_building_id),
                    cluster_type: 'APARTMENT_COMPLEX',
                    member_count: size(apartments),
                    lv_group_id: lv_group,
                    district_name: district,
                    created_at: datetime(),
                    pattern: 'VERTICAL',
                    thermal_benefit: 'LOW',
                    cable_savings: 'VERY_HIGH',
                    avg_shared_walls: 1.0
                })
                
                WITH ac, apartments
                UNWIND apartments as apt
                CREATE (apt)-[:IN_ADJACENCY_CLUSTER {
                    joined_at: datetime()
                }]->(ac)
                
                RETURN count(DISTINCT ac) as created
            """)
            
            apt_result = result.single()
            apt_count = apt_result['created'] if apt_result else 0
            logger.info(f"Created {apt_count} apartment clusters")
            
            total_clusters = row_count + corner_count + apt_count
            self.stats['clusters_created'] = total_clusters
            
            logger.info(f"Created {total_clusters} adjacency clusters total")
            
            return total_clusters
    
    def enhance_building_metrics(self):
        """Add adjacency-based metrics to buildings"""
        logger.info("Enhancing building metrics with adjacency data...")
        
        with self.driver.session() as session:
            # Add adjacency counts and metrics
            session.run("""
                MATCH (b:Building)
                OPTIONAL MATCH (b)-[adj:ADJACENT_TO]-()
                WITH b, 
                     count(DISTINCT adj) as adjacency_count,
                     avg(adj.adjacency_strength) as avg_strength,
                     max(adj.complementarity_potential) as max_complementarity,
                     collect(DISTINCT adj.wall_pair) as shared_walls
                
                SET b.adjacency_count = adjacency_count,
                    b.avg_adjacency_strength = COALESCE(avg_strength, 0),
                    b.max_complementarity = COALESCE(max_complementarity, 0),
                    b.has_adjacent_neighbors = adjacency_count > 0,
                    b.shared_wall_directions = shared_walls,
                    b.isolation_factor = CASE 
                        WHEN adjacency_count = 0 THEN 1.0
                        WHEN adjacency_count = 1 THEN 0.7
                        WHEN adjacency_count = 2 THEN 0.5
                        ELSE 0.3
                    END,
                    b.thermal_efficiency_boost = 1.0 + (adjacency_count * 0.15)
            """)
            
            # Add complementarity potential for adjacent pairs
            session.run("""
                MATCH (b1:Building)-[adj:ADJACENT_TO]-(b2:Building)
                WHERE b1.ogc_fid < b2.ogc_fid
                
                WITH b1, b2, adj,
                     adj.function_diversity * adj.solar_diversity * adj.match_quality as potential
                
                SET adj.complementarity_potential = potential,
                    adj.priority_for_sharing = CASE
                        WHEN potential > 2.5 THEN 'VERY_HIGH'
                        WHEN potential > 2.0 THEN 'HIGH'
                        WHEN potential > 1.5 THEN 'MEDIUM'
                        ELSE 'LOW'
                    END,
                    adj.thermal_resistance_reduction = 0.1 * adj.match_quality
            """)
            
            # Update cluster metrics
            session.run("""
                MATCH (ac:AdjacencyCluster)<-[:IN_ADJACENCY_CLUSTER]-(b:Building)
                WITH ac, 
                     collect(b) as members,
                     avg(b.solar_capacity_kwp) as avg_solar_potential,
                     sum(CASE WHEN b.has_solar THEN 1 ELSE 0 END) as solar_count,
                     sum(CASE WHEN b.has_battery THEN 1 ELSE 0 END) as battery_count,
                     sum(CASE WHEN b.has_heat_pump THEN 1 ELSE 0 END) as hp_count,
                     collect(DISTINCT b.building_function) as functions
                     
                SET ac.avg_solar_potential_kwp = avg_solar_potential,
                    ac.solar_penetration = toFloat(solar_count) / size(members),
                    ac.battery_penetration = toFloat(battery_count) / size(members),
                    ac.hp_penetration = toFloat(hp_count) / size(members),
                    ac.function_diversity = size(functions),
                    ac.energy_sharing_potential = CASE
                        WHEN size(functions) > 1 AND solar_count > 0 THEN 'HIGH'
                        WHEN solar_count > 0 OR battery_count > 0 THEN 'MEDIUM'
                        ELSE 'LOW'
                    END
            """)
            
            logger.info("Building metrics enhanced with adjacency data")
    
    def generate_report(self):
        """Generate comprehensive adjacency report"""
        logger.info("\n" + "="*60)
        logger.info("ADJACENCY UPDATE REPORT")
        logger.info("="*60)
        
        with self.driver.session() as session:
            # Overall statistics
            result = session.run("""
                MATCH (b:Building)
                RETURN 
                    count(b) as total_buildings,
                    sum(CASE WHEN b.num_shared_walls > 0 THEN 1 ELSE 0 END) as with_shared_walls,
                    sum(CASE WHEN b.adjacency_count > 0 THEN 1 ELSE 0 END) as with_adjacencies,
                    avg(b.adjacency_count) as avg_adjacencies
            """)
            stats = result.single()
            
            if stats:
                print(f"\nBuilding Statistics:")
                print(f"  Total buildings: {stats['total_buildings']}")
                print(f"  With shared walls: {stats['with_shared_walls']}")
                print(f"  With adjacencies found: {stats['with_adjacencies']}")
                if stats['avg_adjacencies']:
                    print(f"  Average adjacencies: {stats['avg_adjacencies']:.2f}")
            
            # Adjacency type distribution
            result = session.run("""
                MATCH (b:Building)
                WHERE b.adjacency_type IS NOT NULL
                RETURN b.adjacency_type as type, count(b) as count
                ORDER BY count DESC
            """)
            
            print(f"\nAdjacency Type Distribution:")
            for record in result:
                print(f"  {record['type']}: {record['count']}")
            
            # Housing type analysis - FIXED
            result = session.run("""
                MATCH (b:Building)
                WHERE b.woningtype IS NOT NULL
                OPTIONAL MATCH (b)-[adj:ADJACENT_TO]-()
                WITH b, b.woningtype as housing_type, count(adj) as adjacency_count
                WITH housing_type, 
                    count(DISTINCT b) as count,
                    avg(adjacency_count) as avg_adjacencies
                RETURN housing_type, count, avg_adjacencies
                ORDER BY count DESC
            """)
            
            print(f"\nHousing Type Adjacency Analysis:")
            for record in result:
                if record['avg_adjacencies'] is not None:
                    print(f"  {record['housing_type']}: {record['count']} buildings, "
                        f"avg {record['avg_adjacencies']:.1f} adjacencies")
                else:
                    print(f"  {record['housing_type']}: {record['count']} buildings, no adjacencies")
            
            # Relationship statistics
            result = session.run("""
                MATCH ()-[adj:ADJACENT_TO]->()
                RETURN 
                    count(adj)/2 as total_relationships,
                    avg(adj.distance_m) as avg_distance,
                    avg(adj.adjacency_strength) as avg_strength,
                    avg(adj.match_quality) as avg_match_quality,
                    avg(adj.complementarity_potential) as avg_potential
            """)
            rel_stats = result.single()
            
            print(f"\nAdjacency Relationships:")
            if rel_stats and rel_stats['total_relationships']:
                print(f"  Total pairs: {rel_stats['total_relationships']}")
                if rel_stats['avg_distance']:
                    print(f"  Avg distance: {rel_stats['avg_distance']:.2f} m")
                if rel_stats['avg_strength']:
                    print(f"  Avg strength: {rel_stats['avg_strength']:.2f}")
                if rel_stats['avg_match_quality']:
                    print(f"  Avg match quality: {rel_stats['avg_match_quality']:.2f}")
                if rel_stats['avg_potential']:
                    print(f"  Avg complementarity: {rel_stats['avg_potential']:.2f}")
            
            # Cluster statistics
            result = session.run("""
                MATCH (ac:AdjacencyCluster)
                RETURN 
                    ac.cluster_type as type,
                    count(ac) as count,
                    avg(ac.member_count) as avg_size,
                    avg(ac.solar_penetration) as avg_solar_pen
                ORDER BY count DESC
            """)
            
            print(f"\nAdjacency Clusters:")
            has_clusters = False
            for record in result:
                has_clusters = True
                solar_pen_str = f"{record['avg_solar_pen']:.1%}" if record['avg_solar_pen'] is not None else "N/A"
                print(f"  {record['type']}: {record['count']} clusters, "
                    f"avg size: {record['avg_size']:.1f}, "
                    f"solar pen: {solar_pen_str}")
            if not has_clusters:
                print("  No clusters created")
            
            # LV Group adjacency summary
            result = session.run("""
                MATCH (b:Building)-[:ADJACENT_TO]-()
                WITH b.lv_group_id as lv_group, count(DISTINCT b) as adjacent_buildings
                MATCH (b2:Building {lv_group_id: lv_group})
                WITH lv_group, 
                    adjacent_buildings,
                    count(DISTINCT b2) as total_buildings
                RETURN lv_group,
                    adjacent_buildings,
                    total_buildings,
                    toFloat(adjacent_buildings) / total_buildings as adjacency_ratio
                ORDER BY adjacency_ratio DESC
                LIMIT 10
            """)
            
            print(f"\nTop LV Groups by Adjacency Ratio:")
            for record in result:
                print(f"  {record['lv_group']}: {record['adjacent_buildings']}/{record['total_buildings']} "
                    f"({record['adjacency_ratio']:.1%})")
            
            # Top complementarity pairs
            result = session.run("""
                MATCH (b1:Building)-[adj:ADJACENT_TO]-(b2:Building)
                WHERE b1.ogc_fid < b2.ogc_fid
                AND adj.complementarity_potential > 1.5
                RETURN 
                    b1.ogc_fid as building1,
                    b2.ogc_fid as building2,
                    b1.building_function as func1,
                    b2.building_function as func2,
                    b1.woningtype as type1,
                    b2.woningtype as type2,
                    adj.wall_pair as walls,
                    adj.complementarity_potential as potential
                ORDER BY potential DESC
                LIMIT 5
            """)
            
            print(f"\nTop Complementarity Pairs:")
            has_pairs = False
            for record in result:
                has_pairs = True
                print(f"  Buildings {record['building1']}-{record['building2']}: "
                    f"{record['func1']}/{record['func2']}, "
                    f"{record['type1']}/{record['type2']}, "
                    f"potential: {record['potential']:.2f}")
            if not has_pairs:
                print("  No high-complementarity pairs found")
            
            # Validation results
            if self.stats['validation_results']:
                print(f"\nValidation Results:")
                for key, val in self.stats['validation_results'].items():
                    if key == 'reciprocal':
                        print(f"  {key}: {val.get('reciprocal', 0)}/{val.get('total', 0)} "
                            f"({val.get('accuracy', 0):.1%} accuracy)")
                    else:
                        print(f"  {key}: {val.get('correct', 0)}/{val.get('total', 0)} "
                            f"({val.get('accuracy', 0):.1%} accuracy, "
                            f"avg: {val.get('avg_neighbors', 0):.1f})")
            
            print("\n" + "="*60)
            print("ADJACENCY UPDATE COMPLETE")
            print("="*60)
    
    def run_full_update(self):
        """Run complete adjacency update process using PostgreSQL data"""
        start_time = datetime.now()
        
        try:
            # Check KG status
            status = self.check_kg_status()
            logger.info(f"KG Status: {status}")
            
            if not status['buildings_exist']:
                logger.error("No buildings found in KG. Run main KG builder (Part 1) first.")
                return False
            
            if status['adjacencies_exist']:
                logger.warning("Adjacencies already exist. They will be recreated.")
            
            # Update shared wall data from PostgreSQL
            if not status['has_shared_walls']:
                logger.info("Updating shared wall data from PostgreSQL...")
                self.update_shared_wall_data_from_postgres()
            else:
                logger.info("Shared wall data already exists in Neo4j")
            
            # Create adjacency relationships
            self.create_adjacency_relationships()
            
            # Validate
            validation = self.validate_adjacencies()
            
            # Create clusters
            self.create_adjacency_clusters()
            
            # Enhance metrics
            self.enhance_building_metrics()
            
            # Generate report
            self.generate_report()
            
            # Calculate time
            elapsed = (datetime.now() - start_time).total_seconds()
            logger.info(f"Total processing time: {elapsed:.2f} seconds")
            
            return True
            
        except Exception as e:
            logger.error(f"Error during adjacency update: {str(e)}")
            import traceback
            traceback.print_exc()
            raise

# ============================================
# MAIN EXECUTION
# ============================================

if __name__ == "__main__":
    # Configuration (SAME AS PART 1)
    NEO4J_URI = "bolt://localhost:7687"
    NEO4J_USER = "neo4j"
    NEO4J_PASSWORD = "aminasad"
    
    # PostgreSQL configuration (SAME AS PART 1)
    PG_HOST = "localhost"
    PG_DATABASE = "research"
    PG_USER = "aminj"
    PG_PASSWORD = "Aminej@geodan!"
    
    print("Starting Adjacency Module (Part 2) using PostgreSQL data...")
    print(f"Connecting to PostgreSQL database: {PG_DATABASE}")
    print(f"Connecting to Neo4j at: {NEO4J_URI}")
    
    # Create updater with both connections
    updater = AdjacencyUpdater(
        NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD,
        PG_HOST, PG_DATABASE, PG_USER, PG_PASSWORD
    )
    
    try:
        # Run full update using PostgreSQL data
        success = updater.run_full_update()
        
        if success:
            print("\n✅ Adjacency update completed successfully!")
            print("Your KG now has:")
            print("  - ADJACENT_TO relationships with energy implications")
            print("  - Natural adjacency clusters (row houses, corners, apartments)")
            print("  - Enhanced complementarity potential for adjacent buildings")
            print("  - Thermal coupling metrics")
            print("  - Validation against housing types (woningtype)")
            print("\nReady for GNN processing with adjacency-aware features!")
        
    finally:
        updater.close()

2025-08-18 03:28:21,132 - INFO - Connected to Neo4j and PostgreSQL for adjacency update


Starting Adjacency Module (Part 2) using PostgreSQL data...
Connecting to PostgreSQL database: research
Connecting to Neo4j at: bolt://localhost:7687


2025-08-18 03:28:23,242 - INFO - KG Status: {'buildings_exist': True, 'has_shared_walls': True, 'adjacencies_exist': False, 'building_count': 1185}
2025-08-18 03:28:23,243 - INFO - Shared wall data already exists in Neo4j
2025-08-18 03:28:23,243 - INFO - Creating adjacency relationships...
2025-08-18 03:28:23,719 - INFO - Created 514 adjacency relationships (257 pairs)
2025-08-18 03:28:23,719 - INFO - Validating adjacency patterns...
2025-08-18 03:28:23,745 - INFO - Creating natural adjacency clusters...
2025-08-18 03:28:23,788 - INFO - Creating row house clusters...
2025-08-18 03:28:24,021 - INFO - Created 327 row house clusters
2025-08-18 03:28:24,022 - INFO - Creating corner/courtyard clusters...
2025-08-18 03:28:24,065 - INFO - Created 0 corner/courtyard clusters
2025-08-18 03:28:24,066 - INFO - Creating apartment clusters...
2025-08-18 03:28:24,283 - INFO - Created 0 apartment clusters
2025-08-18 03:28:24,283 - INFO - Created 327 adjacency clusters total
2025-08-18 03:28:24,284 - 


Building Statistics:
  Total buildings: 1517
  With shared walls: 1229
  With adjacencies found: 332
  Average adjacencies: 0.68

Adjacency Type Distribution:
  MIDDLE_ROW: 823
  END_ROW: 330
  ISOLATED: 288
  CORNER: 76

Housing Type Adjacency Analysis:
  unknown: 848 buildings, avg 1.2 adjacencies
  Appartement: 288 buildings, avg 0.0 adjacencies
  Tussen of geschakelde woning: 272 buildings, avg 0.1 adjacencies
  Hoekwoning: 68 buildings, avg 0.0 adjacencies
  Vrijstaande woning: 36 buildings, avg 0.0 adjacencies
  Tweeonder1kap: 5 buildings, avg 0.0 adjacencies

Adjacency Relationships:
  Total pairs: 257
  Avg distance: 3.12 m
  Avg strength: 6.77
  Avg match quality: 1.00
  Avg complementarity: 1.00

Adjacency Clusters:
  ROW_HOUSES: 327 clusters, avg size: 6.8, solar pen: 0.3%


2025-08-18 03:28:24,732 - INFO - Total processing time: 3.60 seconds
2025-08-18 03:28:24,733 - INFO - Connections closed



Top LV Groups by Adjacency Ratio:
  LV_GROUP_0002: 4/5 (80.0%)
  LV_GROUP_0024: 9/12 (75.0%)
  LV_GROUP_0021: 7/11 (63.6%)
  LV_GROUP_0026: 3/5 (60.0%)
  LV_GROUP_0023: 4/7 (57.1%)
  LV_GROUP_0127: 5/9 (55.6%)
  LV_GROUP_0140: 6/11 (54.5%)
  LV_GROUP_0022: 6/13 (46.2%)
  LV_GROUP_0120: 10/23 (43.5%)
  LV_GROUP_0119: 5/13 (38.5%)

Top Complementarity Pairs:
  Buildings 4804268-4804269: residential/residential, Tussen of geschakelde woning/Tussen of geschakelde woning, potential: 1.81
  Buildings 4804268-4804269: residential/residential, Tussen of geschakelde woning/Tussen of geschakelde woning, potential: 1.81

Validation Results:
  middle_row_type: 141/823 (17.1% accuracy, avg: 1.2)
  reciprocal: 257/257 (100.0% accuracy)

ADJACENCY UPDATE COMPLETE

✅ Adjacency update completed successfully!
Your KG now has:
  - ADJACENT_TO relationships with energy implications
  - Natural adjacency clusters (row houses, corners, apartments)
  - Enhanced complementarity potential for adjacent buildin